In [1]:
import librosa
import csv
import numpy as np
import pandas as pd
from sklearn.preprocessing import normalize

In [2]:
train=pd.read_csv('train.csv',on_bad_lines='skip')
train=train.dropna()
check=pd.read_csv("check.csv",on_bad_lines='skip')
check=check.dropna()

In [3]:
train.head()

,label,data1,data2,data3,data4,data5,data6,data7,data8,data9,...,data47095,data47096,data47097,data47098,data47099,data47100,data47101,data47102,data47103,data47104
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000610,0.000854,0.000549,0.001007,0.000854,0.000610,0.000885,0.000519,0.001251,0.000427
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.000092,0.000092,-0.000122,-0.000336,-0.000122,-0.000397,-0.000061,-0.000549,0.000580,-0.000305
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.001404,0.001495,0.001312,0.000610,0.001495,0.001343,0.000824,0.001160,0.000488,0.001160
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.002930,0.001923,0.002472,0.002258,0.002045,0.002228,0.001190,0.001831,0.001617,0.001434
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.003021,0.002869,0.002869,0.002960,0.002747,0.003204,0.003052,0.003418,0.003418,0.003052


In [4]:
check.head()

,label,data1,data2,data3,data4,data5,data6,data7,data8,data9,...,data47095,data47096,data47097,data47098,data47099,data47100,data47101,data47102,data47103,data47104
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.000824,-0.000580,-0.001434,-0.001343,-0.001068,-0.001404,-0.001678,-0.001709,-0.001617,-0.001984
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.001495,0.000946,0.000610,0.001373,0.000977,0.001648,0.001892,0.002136,0.002472,0.002380
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000671,0.000244,0.000305,0.000061,0.000031,0.000519,-0.000122,0.001038,0.000336,0.000244
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.000397,-0.001434,-0.000916,-0.001038,-0.000275,-0.000946,-0.001099,-0.000580,-0.000916,-0.000244
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000885,0.001892,0.001404,0.001007,0.001495,0.001312,0.000885,0.001068,0.000641,0.000763


In [5]:
#corner=1 wall=0
train_label=train['label']
check_label=check['label']
del train['label']
del check['label']

In [6]:
train_label.shape

(1379,)

In [7]:
check_label.shape

(420,)

In [8]:
train.shape

(1379, 47104)

In [9]:
check.shape

(420, 47104)

In [10]:
x_rms=[]
x_sumsq=[]
x_onset_env=[]
tr=np.array(train)
ch=np.array(check)
for i in range(0,np.size(tr,0)):
    x_rms=np.append(librosa.feature.rms(y=tr[i], frame_length=4096, hop_length=4000, center=True, pad_mode='reflect'),x_rms)
    window=librosa.filters.get_window(tr[i], Nx=47104)
    x_sumsq=np.append(librosa.filters.window_sumsquare(window,n_frames=1,hop_length=4000,n_fft=47104),x_sumsq)
    x_onset_env = np.append(librosa.onset.onset_strength(y=tr[i], sr=44100, max_size=12),x_onset_env) 
y_rms=[]
y_sumsq=[]
y_onset_env=[]
for i in range(0,np.size(ch,0)):
    y_rms=np.append(librosa.feature.rms(y=ch[i], frame_length=4096, hop_length=4000, center=True, pad_mode='reflect'),y_rms)
    window2=librosa.filters.get_window(ch[i], Nx=47104)
    y_sumsq=np.append(librosa.filters.window_sumsquare(window2,n_frames=1,hop_length=4000,n_fft=47104),y_sumsq)
    y_onset_env = np.append(librosa.onset.onset_strength(y=ch[i], sr=44100, max_size=12),y_onset_env) 

In [15]:
x_rms=(x_rms-x_rms.min())/(x_rms.max()-x_rms.min())
x_sumsq=(x_sumsq-x_sumsq.min())/(x_sumsq.max()-x_sumsq.min())
x_onset_env=(x_onset_env-x_onset_env.min())/(x_onset_env.max()-x_onset_env.min())
y_rms=(y_rms-y_rms.min())/(y_rms.max()-y_rms.min())
y_sumsq=(y_sumsq-y_sumsq.min())/(y_sumsq.max()-y_sumsq.min())
y_onset_env=(y_onset_env-y_onset_env.min())/(y_onset_env.max()-y_onset_env.min())

In [24]:
def padding(array, xx, yy):
    """
    :param array: numpy array
    :param xx: desired height
    :param yy: desirex width
    :return: padded array
    """
    h = array.shape[0]
    w = 0
    a = max((xx - h) // 2,0)
    aa = max(0,xx - a - h)
    b = max(0,(yy - w) // 2)
    bb = max(yy - b - w,0)
    return np.pad(array, pad_width=((a, aa), (b, bb)), mode='constant')

In [25]:
def img (rms, onset, sumsq):
    max_size=47104 #my max audio file feature width
    #Now the padding part
    image = np.array([padding(rms,1,max_size )]).reshape(1,max_size)
    image = np.append(image,padding(sumsq,1, max_size), axis=0) 
    #repeat the padded spec_bw,spec_centroid and chroma stft until they are stft and MFCC-sized
    image = np.append(image,padding(onset,1, max_size), axis=0)
    return image

In [26]:
train=img(x_rms, x_onset_env, x_sumsq)
test= img(y_rms, y_onset_env, y_sumsq)

ValueError: operands could not be broadcast together with remapped shapes [original->remapped]: (2,2)  and requested shape (1,2)

In [15]:
osm.max()

1.0

In [16]:
wow.max()

1.0

In [12]:
CNNmodel = model.Sequential()
CNNmodel.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
CNNmodel.add(layers.MaxPooling2D((2, 2)))
CNNmodel.add(layers.Dropout(0.2))
CNNmodel.add(layers.Conv2D(64, (3, 3), activation='relu'))
CNNmodel.add(layers.MaxPooling2D((2, 2)))
CNNmodel.add(layers.Dropout(0.2))
CNNmodel.add(layers.Conv2D(64, (3, 3), activation='relu'))
CNNmodel.add(layers.Flatten())
CNNmodel.add(layers.Dense(64, activation='relu'))
CNNmodel.add(layers.Dropout(0.2))
CNNmodel.add(layers.Dense(32, activation='relu'))
CNNmodel.add(layers.Dense(24, activation='softmax'))

NameError: name 'models' is not defined

In [ ]:
model.summary()

In [ ]:
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),metrics=['accuracy'],optimizer='adam')

In [ ]:
model.fit(osm,train_label,epochs=30,verbose=1)

In [ ]:
model.evaluate(wow,check_label)